In [1]:
from aocd import get_data

Given a 2D grid representing antennas with specific frequencies (letters, digits), identify locations called "antinodes" where signals resonate.

Rules for Antinodes:

**Same Frequency:** Two antennas must have the same frequency for there to be antinodes

**Collinearity:** The antennas and antinodes lie on a straight line

**Resonance Condition:** One antenna must be twice as far from the antinode as the other

**Antinode Count:** Each valid antenna pair generates two antinodes (one on each side)

**Objective:** Count the unique antinode locations within the grid bounds

In [2]:
example_input = """
............
........0...
.....0......
.......0....
....0.......
......A.....
............
............
........A...
.........A..
............
............
"""

In [4]:
import numpy as np

In [5]:
example_grid = np.array([list(line) for line in example_input.strip().split("\n")])

In [6]:
example_grid

array([['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '0', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '0', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '0', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '0', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', 'A', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', 'A', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', 'A', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']],
      dtype='<U1')

Identify the positions of all antennas with their corresponding "frequency" values

In [34]:
def parse_grid(grid):

    # boolean mask for alphanumeric elements
    mask = np.char.isalnum(grid)
    
    positions = np.argwhere(mask)
    antenna_values = grid[mask]
    
    antennas = np.array([(i, j, val) for (i, j), val in zip(positions, antenna_values)], dtype=object)
    
    return antennas

In [35]:
antennas = parse_grid(example_grid); antennas

array([[np.int64(1), np.int64(8), np.str_('0')],
       [np.int64(2), np.int64(5), np.str_('0')],
       [np.int64(3), np.int64(7), np.str_('0')],
       [np.int64(4), np.int64(4), np.str_('0')],
       [np.int64(5), np.int64(6), np.str_('A')],
       [np.int64(8), np.int64(8), np.str_('A')],
       [np.int64(9), np.int64(9), np.str_('A')]], dtype=object)

- Loop through all possible pairs of antennas that have the same frequency
- For each pair, find the potential antinode locations such that one antenna is twice as far from the antinode as the other
- Ensure that the calculated antinode locations are within the bounds of the map
- Use a `set` to store and deduplicate the valid antinode locations
- Return the number of unique locations stored in the set

### First Approach

In [27]:
def within_bounds(node, grid):
    return 0 <= node[0] < len(grid) and 0 <= node[1] < len(grid[0])

In [28]:
antinodes = set()
    
for i in range(len(antennas)):
    for j in range(i + 1, len(antennas)):
        x1, y1, val1 = antennas[i]
        x2, y2, val2 = antennas[j]
        
        if val1 != val2:
            continue
        
        # deltas for calculating antinode locations
        dx, dy = x2 - x1, y2 - y1
        
        # potential antinode locations
        antinode1 = (x1 - dx, y1 - dy)
        antinode2 = (x2 + dx, y2 + dy)
        
        # add valid antinodes that are within bounds
        if within_bounds(antinode1, example_grid):
            antinodes.add(antinode1)
        if within_bounds(antinode2, example_grid):
            antinodes.add(antinode2)

In [29]:
len(antinodes)

14

### Vecotorized

In [38]:
def find_antinodes(antennas, grid_shape):
    
    coords = antennas[:, :2]
    freq_values = antennas[:, -1]
    
    # create all unique pairs of indices (i, j) where i < j with this one weird trick
    i_indices, j_indices = np.triu_indices(len(antennas), k=1)
    
    coords1 = coords[i_indices]
    coords2 = coords[j_indices]
    freqs1 = freq_values[i_indices]
    freqs2 = freq_values[j_indices]
    
    matching_pairs = freqs1 == freqs2
    
    dx_dy = coords2[matching_pairs] - coords1[matching_pairs]
    
    # potential antinodes
    antinode1 = coords1[matching_pairs] - dx_dy
    antinode2 = coords2[matching_pairs] + dx_dy
    
    def within_bounds(coords, shape):
        return np.all((coords >= 0) & (coords < shape), axis=1)
    
    valid_antinode1 = antinode1[within_bounds(antinode1, grid_shape)]
    valid_antinode2 = antinode2[within_bounds(antinode2, grid_shape)]
    
    # combine and remove duplicates by converting to set of tuples
    antinodes = set(map(tuple, valid_antinode1)) | set(map(tuple, valid_antinode2))
    
    return antinodes


In [39]:
len(find_antinodes(antennas, example_grid.shape))

14

In [42]:
data = get_data(day=8, year=2024)

In [43]:
grid = np.array([list(line) for line in data.strip().split("\n")])

In [47]:
antennas = parse_grid(grid); antennas[:10]

array([[np.int64(0), np.int64(21), np.str_('5')],
       [np.int64(0), np.int64(37), np.str_('P')],
       [np.int64(1), np.int64(13), np.str_('w')],
       [np.int64(1), np.int64(19), np.str_('T')],
       [np.int64(1), np.int64(29), np.str_('X')],
       [np.int64(1), np.int64(30), np.str_('h')],
       [np.int64(1), np.int64(36), np.str_('5')],
       [np.int64(1), np.int64(49), np.str_('u')],
       [np.int64(2), np.int64(19), np.str_('k')],
       [np.int64(2), np.int64(20), np.str_('X')]], dtype=object)

In [46]:
len(find_antinodes(antennas, grid.shape))

400

## Part Two

Antinodes are not only created by pairs of antennas that are in line with each other but also by the positions of the antennas themselves—if there is more than one antenna with the same frequency.

**Antennas can now be antinodes:** If an antenna shares its frequency with at least one other antenna, its position becomes an antinode.

**New antinode locations:** For each pair of antennas with the same frequency, antinodes will be generated not only between them (based on distance) but also at the antennas' positions if there are multiple antennas of the same frequency.

**Counting Unique Antinodes:** The goal is to count all unique positions that are antinodes, including those generated by overlapping antennas of the same frequency.